In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

In [2]:
# %% 기준일과 매칭할 날짜 수 setting
daycount_match = 2 # 매칭할 대상 기간 : 2 days
daycount_forecast = 1 # 전망할 날짜 수 :  1 day
resample_minutes = 10 # 가격주기 : 10 mins

date_split = '2019-01-01'

In [4]:
# %% 1min 단위의 raw data 받아오기
df_infomax = pd.read_csv("../data/LKTB_infomax_1m.csv")
df_infomax['DATETIME'] = pd.to_datetime(df_infomax['DATE'] + ' ' + df_infomax['TIME'])
df = df_infomax.drop(['DATE', 'TIME'], axis=1)
df = df.set_index('DATETIME')

In [5]:
# %% resample
df = df.resample(str(resample_minutes) + 'T', closed='right').agg({
    'OPEN': 'first',
    'HIGH': 'max',
    'LOW': 'min',
    'CLOSE': 'last',
    'VOLUME': 'sum',
    'OPENINTEREST': 'last'
    }).dropna()

In [6]:
# %% df 2019와 이전으로 나누기
df_data = df[:date_split]
df_match = df[date_split:'2019-02-28']

# In[14]: 날짜 list 만들기
list_dates_data = df_data.resample('D') \
    .last() \
        .dropna() \
            .drop(['OPEN', 'HIGH', 'LOW', 'VOLUME', 'OPENINTEREST'], axis=1) \
                .reset_index()['DATETIME'] \
                    .dt.date \
                        .to_numpy().tolist()

list_dates_match = df_match.resample('D') \
    .last() \
        .dropna() \
            .drop(['OPEN', 'HIGH', 'LOW', 'VOLUME', 'OPENINTEREST'], axis=1) \
                .reset_index()['DATETIME'] \
                    .dt.date \
                        .to_numpy().tolist()

In [8]:
# %%
list_data_date1 = []
list_data_date2 = []
list_data_date3 = []
list_data_date4 = []
for i in range(len(list_dates_data) - daycount_match - daycount_forecast + 1 ):
    list_data_date1.append(list_dates_data[i])
    list_data_date2.append(list_dates_data[i + daycount_match - 1] )
    list_data_date3.append(list_dates_data[i + daycount_match] )
    list_data_date4.append(list_dates_data[i + daycount_match + daycount_forecast - 1] )

list_match_date1 = []
list_match_date2 = []
list_match_date3 = []
list_match_date4 = []
for i in range(len(list_dates_match) - daycount_match - daycount_forecast + 1):
    list_match_date1.append(list_dates_match[i])
    list_match_date2.append(list_dates_match[i + daycount_match - 1])
    list_match_date3.append(list_dates_match[i + daycount_match] )
    list_match_date4.append(list_dates_match[i + daycount_match + daycount_forecast - 1] )


In [9]:
# %% window 별 moves 분리
list_data_moves = []
list_match_moves = []
list_price_chg_actual = []
for i in range(len(list_data_date1)):
    list_data_moves.append(df_data.loc[list_data_date1[i].isoformat():list_data_date2[i].isoformat()]['CLOSE'].to_numpy())
for i in range(len(list_match_date1)):
    list_match_moves.append(df_match.loc[list_match_date1[i].isoformat():list_match_date2[i].isoformat()]['CLOSE'].to_numpy())
    list_price_chg_actual.append(df_match.loc[list_match_date4[i].isoformat()]['CLOSE'].to_numpy()[-1] - list_match_moves[i][-1])

In [10]:
# %% for문 돌며 dtw 계산
#list_dtw = []

from dtw import accelerated_dtw
from tqdm import tqdm
max_rank = 20
list_price_chg_forecast = []

for np_moves_match in tqdm(list_match_moves): # match_moves의 각 하루치 일자 당 반복하며
    list_dtw = []
    price_chg_forecast = 0
    for np_moves_data in (list_data_moves):
        d, _, _, _ = accelerated_dtw(stats.zscore(np_moves_match), stats.zscore(np_moves_data), 'euclidean')
        list_dtw.append(d)
        list_rank = stats.rankdata(list_dtw).tolist()
    
    sum_chg_mult_weight = 0
    sum_weight = 0    
    for rank in range(1,max_rank+1):
        index = list_rank.index(rank)
        weight = 1/list_dtw[index]
        #date3 = list_data_date3[index]
        date4 = list_data_date4[index]
        pivot_price = list_data_moves[index][-1]
        close_price = df_data.loc[date4.isoformat()]['CLOSE'].to_numpy()[-1]
        price_chg = close_price - pivot_price
        sum_chg_mult_weight += price_chg * weight
        sum_weight += weight
    
    price_chg_forecast = sum_chg_mult_weight / sum_weight
    list_price_chg_forecast.append(price_chg_forecast)

100%|██████████| 37/37 [13:11<00:00, 21.38s/it]


In [11]:
# %% result dataframe 만들기
df_result = pd.DataFrame(list(zip(list_match_date1,
                                  list_match_date2,
                                  list_match_date3,
                                  list_match_date4,
                                  list_price_chg_forecast,
                                  list_price_chg_actual)
                              ),
                         columns = ['date1', 'date2', 'date3', 'date4', 'forecast', 'actual'])

In [12]:
df_result

,date1,date2,date3,date4,forecast,actual
0,2019-01-02,2019-01-03,2019-01-04,2019-01-04,0.069398,-0.07
1,2019-01-03,2019-01-04,2019-01-07,2019-01-07,0.045506,-0.15
2,2019-01-04,2019-01-07,2019-01-08,2019-01-08,0.009678,-0.18
3,2019-01-07,2019-01-08,2019-01-09,2019-01-09,0.052750,0.05
4,2019-01-08,2019-01-09,2019-01-10,2019-01-10,-0.058584,0.22
5,2019-01-09,2019-01-10,2019-01-11,2019-01-11,-0.090385,-0.27
6,2019-01-10,2019-01-11,2019-01-14,2019-01-14,0.084895,0.35
7,2019-01-11,2019-01-14,2019-01-15,2019-01-15,0.061312,0.10
8,2019-01-14,2019-01-15,2019-01-16,2019-01-16,0.061831,-0.10
9,2019-01-15,2019-01-16,2019-01-17,2019-01-17,0.182793,-0.05
